# Interacting with LLMs from various providers using Haystack 

![](./pipeline-diagrams/dag.jpg)

Note: if you are running this notebook using Google colab, a requirements.txt file has been prepared with the following dependencies:

```bash
transformers==4.34.1
haystack-ai==0.93.0
cohere==4.31
tiktoken==0.5.1
python-dotenv
```

Upload the requirements_colab.txt file onto your Google Colab session. You can install them using the following command:

```bash
!pip install -r requirements_colab.txt
```

## Connecting to OpenAI's GPT models

Ensure you create an OpenAI account and set the OPENAI_API_KEY environment variable. You can do this by creating a .env file in the same directory as this notebook and adding the following line:

```bash
OPENAI_API_KEY=<your-api-key>
```

In [3]:
from haystack.preview.components.generators.openai import GPTGenerator
from haystack.preview.components.generators.hugging_face_local import HuggingFaceLocalGenerator
import os
from dotenv import load_dotenv
import sys

# Get the current and parent working directory
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Append parent directory to sys.path
sys.path.append(parent_directory)
from scripts.pipelines import initialize_simple_pipeline

In [3]:
load_dotenv()
# Applicable only if your env file is stored two levels above the current directory
load_dotenv("./../../.env")


try:
    # Open AI API Key
    openai_key = os.getenv("OPENAI_API_KEY")

    if openai_key==None:
        raise Exception("Please set OPENAI_API_KEY as an environment variable")
except Exception as e:
    print("Please set OPENAI_API_KEY as an environment variable")
    print(e)

In [5]:
gpt = GPTGenerator(api_key=os.getenv("OPENAI_API_KEY"),  
                                                  model_name='gpt-4', 
                                                  api_base_url = 'https://api.openai.com/v1',
                                                  )

In [6]:
gpt.run("You are a helpful assistant who provides answers to questions. What is the capital of France?")

{'replies': ['The capital of France is Paris.'],
 'metadata': [{'model': 'gpt-4-0613',
   'index': 0,
   'finish_reason': 'stop',
   'usage': {'prompt_tokens': 25,
    'completion_tokens': 7,
    'total_tokens': 32}}]}

In [4]:
prompt_template = """
        Given a question, you provide a short answer.
        \nQuestion: {{question}}
        \nAnswer:
        """


In [ ]:

# Initializing the pipeline
simple_pipeline = initialize_simple_pipeline(llm_generator=gpt, llm_generator_name="gpt_generator", prompt_template=prompt_template)

# Running a question through the pipeline
question = "What is the capital of France?"
result = simple_pipeline.run(
    {
        "prompt_builder": {"question": question},
    }
)

# Printing the result
print(result)


In [ ]:
simple_pipeline.draw( "pipeline-diagrams/gpt-pipeline.png")


## Connecting to Local HuggingFace LLM

Important: the code below will download a local copy of the model on your local computer. Running the code below can be time consuming and will require GPU and RAM resources.

You can use Google Colab to run this notebook. It is recommended that you use a Pro account with a T4 GPU runtime enabled. 

In [ ]:
local_generator = HuggingFaceLocalGenerator(model_name_or_path="bigscience/bloom-1b",
                                          task="text-generation",
                                          generation_kwargs={
                                            "max_new_tokens": 1000,
                                            "temperature": 0.1,
                                            "do_sample":  True
                                            })

In [ ]:
local_generator.warm_up()

In [ ]:
answer = local_generator.run("Who lives in Paris?")

In [ ]:
hf_pipeline = initialize_simple_pipeline(llm_generator=local_generator, 
                                          llm_generator_name="hf_generator", 
                                          prompt_template=prompt_template)

In [ ]:
question = "You are a helpful assistant who provides answers to questions. What is the capital of France?"
result = hf_pipeline.run(
    {
        "prompt_builder": {"question": question},
    }
)

# Printing the result
print(result)

In [ ]:
hf_pipeline.draw( "pipeline-diagrams/hf-pipeline.png")


## Custom component: connecting to Mistral using a custom component

In [ ]:
from scripts.huggingfaceendpoints import InferenceEndpointAPI

In [ ]:
hugging_face_token_endpoint = os.getenv("mistral_hf_token")
hf_url = os.getenv("mistral_hf_endpoint")

mistral = InferenceEndpointAPI(api_url=hf_url, api_key=hugging_face_token_endpoint, parameters={
    "temperature": 0.01,
    "max_length": 100
})

mistral.run("Answer using one word: What is the Capital of France")

In [ ]:

mistral_pipeline = initialize_simple_pipeline(llm_generator=mistral, 
                                          llm_generator_name="mistral_generator", 
                                          prompt_template=prompt_template)

In [ ]:
mistral_pipeline.draw( "pipeline-diagrams/mistral-pipeline.png")

In [ ]:
question = "What is the capital of Rome?"
result = mistral_pipeline.run(
    {
        "prompt_builder": {"question": question},
    }
)

# Printing the result
print(result)
